<a href="https://colab.research.google.com/github/karsarobert/ChatGpt_2024/blob/main/ChatGPT2024_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chat GPT és más nagy nyelvi modellek alkalmazása
##PTE Gépi tanulás
###9. gyakorlat: Langchain 2. rész
2024. április 22.

SerpAPI-hoz API kulcs beszerzése
https://serpapi.com



In [ ]:
!pip install openai langchain serpapi google-search-results pypdf tiktoken faiss-cpu

#ReAct (érvelés, cselekvés)
2022-ben Yao és kollégái egy ReAct névre keresztelt új megközelítést dolgoztak ki, amely nagy nyelvi modelleket (LLM-eket) használ gondolkodási folyamatok lekövetésére és feladat-specifikus tevékenységek egyidejű végrehajtására. Ez a technika lehetővé teszi a modell számára, hogy akcióterveket alkosson, ezeket nyomon kövesse és felülvizsgálja, valamint hogy hatékonyan kezelje a váratlan szituációkat. A megközelítés magában foglal egy olyan mechanizmust, amely kölcsönhatásba lép külső elemekkel és adatokat szerez azoktól, mint amilyenek például tudásbázisok vagy más környezetek.

Ez a módszer jelentősen javítja az LLM-ek képességét kiegészítő eszközökkel való együttműködésre, így precízebb, tényeken alapuló válaszokat biztosít. A ReAct hatékonysága a nyelvi feldolgozási és döntéshozatali feladatokban jól látható, mivel több jelenlegi vezető módszert is felülmúl. Ezenkívül a ReAct emberi szemszögből nézve növeli az LLM-ek érthetőségét és megbízhatóságát. A szerzők által azonosított leghatásosabb stratégia ötvözi a ReAct-et a "láncolt gondolkodással" (chain-of-thought, CoT),  igy hasznosítva mind a belső meglátásokat, mind a külső forrásokból származó információkat a gondolkodási folyamat során.

In [ ]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from google.colab import userdata

search = SerpAPIWrapper(serpapi_api_key=userdata.get('SERPAPI_API_KEY'))
tools = [Tool.from_function(
        func=search.run,
        name="Search",
        description="hasznos, ha aktuális eseményekkel kapcsolatos kérdésekre kell válaszolnia"
    )]
agent = initialize_agent(tools, llm = OpenAI(api_key = userdata.get('openai_api_key')), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.invoke("Ki volt Arany János felesége és mikor született?")



> Entering new AgentExecutor chain...
 When dealing with historical questions, it's best to use search engines to find reliable sources.
Action: Search
Action Input: Arany János wife
Observation: Julianna Ercsey
Thought: Now that I have the name, I can search for her date of birth.
Action: Search
Action Input: Julianna Ercsey date of birth
Observation: ['Ercsey. 0 references. date of birth. 1 May 1818 Gregorian. 1 reference. imported from Wikimedia project · Hungarian Wikipedia · place of birth.', 'Julianna Arany (born Ercsey) was born on month day 1818, in birth place, to Sándor Ercsey and Erzsébet Ercsey (born Szakmári).', 'Date of birth/death, circa 1846 Edit this at Wikidata, 1915 Edit this at Wikidata. Work location. Budapest · Edit this at Wikidata · Authority file · wikidata ...', 'Julianna Arany (Ercsey). Birthdate: March 01, 1840. Birthplace: Nagyszalonta, Hungary, Austro-Hungarian Empire.', 'Arany. 0 references. date of birth. 9 August 1841 Gregorian. 1 reference. imported 

{'input': 'Ki volt Arany János felesége és mikor született?',
 'output': 'Julianna Ercsey was born on May 1st, 1818.'}

#Ügynöktípusok

##Zero shot React Description
A "ZERO_SHOT_REACT_DESCRIPTION" ágenstípussal konfigurálva a rendszer csak az eszköz leírása alapján dönt arról, hogy melyik eszközt használja. Ez szemben áll más konfigurációkkal, ahol további bemeneti adatok is felhasználhatók a döntés meghozatalánál.


##Structured input ReAct
Ez egy olyan ágens típus, amely a ReAct keretrendszert használja természetes nyelvű válaszok generálására strukturált bemeneti adatok alapján. Az ágens képes különböző típusú bemeneti adatok kezelésére, például táblázatokra, listákra vagy kulcs-érték párokra. Az ágens egy nyelvi modellt és egy promptot használ, hogy informatív, tömör és összefüggő válaszokat generáljon.

##OpenAI Functions
Ez egy olyan ágenstípus, amely az OpenAI Functions API-t használja a különböző nyelvi modellekhez és az OpenAI eszközeihez való hozzáféréshez. Az ágens különböző funkciókat, például GPT-3, Codex, DALL-E, CLIP vagy ImageGPT használhat. Az ágens egy nyelvi modellt és egy promptot használ, hogy kéréseket generáljon az OpenAI Functions API-hoz, és elemezze a válaszokat.

##Conversational
Ez egy olyan ágens típus, amely nyelvi modellt használ a felhasználóval folytatott természetes nyelvű beszélgetésekhez. Az ágens képes különböző típusú beszélgetési feladatok kezelésére, például csevegésre, kérdés-feleletre vagy feladatok végrehajtására. Az ágens egy nyelvi modellt és egy promptot használ, hogy releváns, gördülékeny és lebilincselő válaszokat hozzon létre.

##Self ask with search
Ez egy olyan ágenstípus, amely egy nyelvi modellt használ arra, hogy kérdéseket generáljon magának, majd válaszokat keressen rájuk a weben. Az ágens ezt a technikát használhatja új információk elsajátítására vagy saját tudásának tesztelésére.

##ReAct document store
Ez egy olyan ágens típus, amely a ReAct keretrendszert használja, hogy természetes nyelvű válaszokat hozzon létre az adatbázisban tárolt dokumentumok alapján. Az ágens különböző típusú dokumentumokat képes kezelni, például hírcikkeket, blogbejegyzéseket vagy kutatási anyagokat.

##Plan-and-execute agents
Ez egy kísérleti ágens típus, amely nyelvi modellt használ, hogy a felhasználó bemenete és egy cél alapján kiválasszon egy végrehajtandó akciósorozatot. Az ágens különböző eszközöket vagy modelleket használhat a kiválasztott akciók végrehajtásához. Az ágens egy nyelvi modellt és egy promptot használ tervek és tevékenységek létrehozásához, majd egy AgentExecutort használ a végrehajtásukhoz.

#Más Huggingfaces modellek használata
Modell hozzáférés kérése a Huggingface-en!!!

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", huggingfacehub_api_token=userdata.get('HF_TOKEN'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
agent_HF = initialize_agent(tools, llm = llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent_HF.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
 I don't have this information, I need to search for the current population of Canada in 2023.
Action: Search
Action Input: Canada population 2023
Observation: 40.77 million
Thought: I now know the final answer
Final Answer: As of 2023, approximately 40.77 million people live in Canada.

> Finished chain.


'As of 2023, approximately 40.77 million people live in Canada.'

#Utazási irodának Chatbot készítése (ConversationChain használata)

In [ ]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(api_key = userdata.get('openai_api_key'))
messages = [
    SystemMessage(content="Ön egy hasznos asszisztens, aki segít a felhasználónak megtervezni egy optimalizált útvonalat."),
    HumanMessage(content="Bécsbe utazom 2 napra mit tudnék megnézni?") ]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
output = chat(messages)
print(output.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Bécs egy gyönyörű és történelmi város, rengeteg látnivalóval. Íme néhány javaslat, hogyan tervezheted meg a két napodat:

**Első nap:**
1. **Schönbrunni Palota és kertjei:** A csodálatos barokk palota és hozzá tartozó kertek Bécs egyik legnépszerűbb látnivalója.
2. **Belváros (Innen a metróval könnyen megközelíthető):** Séta a belvárosban, ahol számos történelmi épületet és nevezetességet találhatsz, például a Szent István Székesegyházat vagy a Hofburg Palotát.
3. **Kávézók és cukrászdák:** Ne hagyd ki a híres bécsi kávézókat és cukrászdákat, például a Café Central-t vagy a Demel-t.

**Második nap:**
1. **Albertina Múzeum:** Gyönyörű műalkotásokat találhatsz itt, köztük festményeket és rajzokat is.
2. **Práter:** Ha szereted a vidámparkokat, akkor látogass el a Práterbe, ahol egyéb szórakozási lehetőségek is várnak rád.
3. **Duna part:** Sétálj a Duna partján, élvezd a kilátást és pihenj egy kicsit a folyó mellett.

Ezek csak néhány javaslat, természetesen Bécs még számos más látnivaló

##Memória hozzáadása:
Tekintve, hogy egy viszonylag rövid üzenetekkel kommunikáló beszélgető botról (chatbotról) van szó, ebben a forgatókönyvben megfelelő lehet egy ConversationBufferMemory használata. A könnyebb konfiguráció érdekében hozzunk létre egy ConversationChain-t is, hogy ötvözzük az LLM (nyelvi modell) és a memória komponenseket. Először inicializáljuk a memóriánkat és a láncunkat (a verbose = True beállítást meghagyom, hogy lássuk, ahogy a bot nyomon követi az előző üzeneteket).

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat, verbose=True, memory=memory
)

In [ ]:
conversation.run("Szia!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Szia!
AI:

> Finished chain.


'Hello! Hogyan vagy?'

In [ ]:
conversation.run("Melyek a fő látványosságok Bécsben?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Szia!
AI: Hello! Hogyan vagy?
Human: Melyek a fő látványosságok Bécsben?
AI:

> Finished chain.


'Bécsben számos fő látványosság található, például a Schoenbrunn Palota és kertje, a Stephansdom katedrális, a Hofburg Császári Palota, valamint a Belvedere Palota. Emellett érdemes ellátogatni a Práter vidámparkba és a Bécsi Állatkertbe is. Bécs híres az operaházáról és a múzeumokról is, mint például a Kunsthistorisches Múzeum és a Albertina Múzeum. Van még sok más látnivaló is, amelyeket érdemes felfedezni!'

In [ ]:
conversation.run("A Práter érdekelne!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Szia!
AI: Hello! Hogyan vagy?
Human: Melyek a fő látványosságok Bécsben?
AI: Bécsben számos fő látványosság található, például a Schoenbrunn Palota és kertje, a Stephansdom katedrális, a Hofburg Császári Palota, valamint a Belvedere Palota. Emellett érdemes ellátogatni a Práter vidámparkba és a Bécsi Állatkertbe is. Bécs híres az operaházáról és a múzeumokról is, mint például a Kunsthistorisches Múzeum és a Albertina Múzeum. Van még sok más látnivaló is, amelyeket érdemes felfedezni!
Human: A Práter érdekelne!
AI:

> Finished chain.


'A Práter egy híres vidámpark Bécsben, amelyet már 1766 óta üzemeltetnek. Az egyik legismertebb attrakciója a Riesenrad nevű óriáskerék, amely csodálatos kilátást nyújt a városra. Emellett számos más izgalmas játék és kaland várja az látogatókat, például hullámvasút, dodgem autók, és különféle játékok. A Práter egy nagyszerű hely a családoknak és a barátoknak egyaránt, és a helyiek és turisták egyaránt szeretik látogatni. Biztosan élvezni fogod!'

In [ ]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Szia!'), AIMessage(content='Hello! Hogyan vagy?'), HumanMessage(content='Melyek a fő látványosságok Bécsben?'), AIMessage(content='Bécsben számos fő látványosság található, például a Schoenbrunn Palota és kertje, a Stephansdom katedrális, a Hofburg Császári Palota, valamint a Belvedere Palota. Emellett érdemes ellátogatni a Práter vidámparkba és a Bécsi Állatkertbe is. Bécs híres az operaházáról és a múzeumokról is, mint például a Kunsthistorisches Múzeum és a Albertina Múzeum. Van még sok más látnivaló is, amelyeket érdemes felfedezni!'), HumanMessage(content='A Práter érdekelne!'), AIMessage(content='A Práter egy híres vidámpark Bécsben, amelyet már 1766 óta üzemeltetnek. Az egyik legismertebb attrakciója a Riesenrad nevű óriáskerék, amely csodálatos kilátást nyújt a városra. Emellett számos más izgalmas játék és kaland várja az látogatókat, például hullámvasút, dodgem autók, és különféle játékok.

In [ ]:
while True:
    query = input('you: ')
    if query == 'q':
        break
    output = conversation({"input": query})
    print('User: ', query)
    print('AI system: ', output['response'])

you: Mesélnél az óriáskerékről?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today?
Human: Szia!
AI: Szia! Magyarul beszélsz?
Human: Melyek a fő látványosságok Bécsben?
AI: Bécsben számos fő látványosság található. Az osztrák főváros híres a Schönbrunn-palotájáról, a Bécsi Operaházból, a Stephansdom katedrálisáról, valamint a Belvedere palotáról. Emellett a városban található a Práter vidámpark, a Hofburg császári palota és a Hundertwasserhaus is. A Bécsi Szimfonikusok és a Bécsi Filharmonikusok koncertjei is nagyon népszerűek. Van valami konkrét, amiről többet szeretnél tudni?
Human: A Práter érdekelne!
AI: A Práter egy híres vidámpark Bécsben, amely 1766 óta műkö

#Vektoros adatbázis hozzáadása a tudástárunkhoz, az LLM paraméteres tudását kiegészítendő

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,
            chunk_overlap=200
        )

raw_documents = PyPDFLoader('https://www.wien.info/resource/blob/412434/49dcd5d77c516144b39195951e771b83/route-wege-die-stadt-kennen-zu-lernen-en-data.pdf#').load()
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings(api_key = userdata.get('openai_api_key')))


memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True
        )
llm = ChatOpenAI(api_key = userdata.get('openai_api_key'))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


##Csak a vektortárból kérdezünk!!!

In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Mi a Mozart ház érdekessége?'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
WestartwiththeMozarthaus Vienna experience .
Itscente rpiec e,Mozart’sopulentoriginal apart-
ment onthegrandBeletageorpianonobile,
hostedmanyofthegreatcomp oser’sparties
betw een1784and1787.And nowtoamodern-
eravenue: anyoneinsearchofjazzinVienna
willstumble upon Porgy&Besssooner orlater,wher ethere’slive
music toenjoyvirtuallyeveryday.Oncealegenda ryeroticcinem a,
todayPorgyhasareputationasthemost beautifuljazzclubin
Europe.Meanwhile ,attheRonacher ,abeacon ofVienna ‘stheater
scene,it‘sshowtime forbigmusicalproduc tions.UpnextisVienna’s
sound museum, theHouse ofMusic .Dullmuseums areathing of
thepast –feelfreetojoininhere:wield theconducto r’sbaton,
experiment with sounds. Agreatplace forthewhole fa

'A Mozart-ház központi eleme Mozart dúsgazdag eredeti lakása a grand Beletage-on vagy pianonobile-on, ahol a nagy zeneszerző sok partit rendezett 1784 és 1787 között. A modernabb vonalon a Porgy & Bess nevű jazzklubot említik, ahol szinte minden nap élő zene szól. A Porgy & Bess ma csodálatos jazzklubként híres Európában.'

##Készítsünk olyan ügynököt amelyik eldönti, hogy használja-e a vektoradatbázist!

In [ ]:
#from langchain.tools.retriever import create_retriever_tool
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    db.as_retriever(),
    "ausztriai_kereso",
    "Ausztriára vonatkozó dokumentumok keresése és visszaküldése.",
)
tools = [tool]
memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=True
        )
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(api_key = userdata.get('openai_api_key'), temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

In [ ]:
agent_executor({"input": "Mesélj nekem a Mozartházról!"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `ausztriai_kereso` with `{'query': 'Mozartház'}`


WestartwiththeMozarthaus Vienna experience .
Itscente rpiec e,Mozart’sopulentoriginal apart-
ment onthegrandBeletageorpianonobile,
hostedmanyofthegreatcomp oser’sparties
betw een1784and1787.And nowtoamodern-
eravenue: anyoneinsearchofjazzinVienna
willstumble upon Porgy&Besssooner orlater,wher ethere’slive
music toenjoyvirtuallyeveryday.Oncealegenda ryeroticcinem a,
todayPorgyhasareputationasthemost beautifuljazzclubin
Europe.Meanwhile ,attheRonacher ,abeacon ofVienna ‘stheater
scene,it‘sshowtime forbigmusicalproduc tions.UpnextisVienna’s
sound museum, theHouse ofMusic .Dullmuseums areathing of
thepast –feelfreetojoininhere:wield theconducto r’sbaton,
experiment with sounds. Agreatplace forthewhole family!
Theshining high light andphot ohotspot intheStadtp arkisthe
gold enJohann Strauss memori al.Thewaltzking freque ntlyper-
formed intheKursalon right nexttoit.Just afewsteps farther
onw

{'input': 'Mesélj nekem a Mozartházról!',
 'chat_history': [HumanMessage(content='Mesélj nekem a Mozartházról!'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Mozartház"}', 'name': 'ausztriai_kereso'}}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 117, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-9b790358-693e-4821-8d69-3c2fdcc5fc73-0'),
  FunctionMessage(content='WestartwiththeMozarthaus Vienna experience .\nItscente rpiec e,Mozart’sopulentoriginal apart-\nment onthegrandBeletageorpianonobile,\nhostedmanyofthegreatcomp oser’sparties\nbetw een1784and1787.And nowtoamodern-\neravenue: anyoneinsearchofjazzinVienna\nwillstumble upon Porgy&Besssooner orlater,wher ethere’slive\nmusic toenjoyvirtuallyeveryday.Oncealegenda ryeroticcinem a,\ntodayPorgyhasareputationasthemost beautifuljazzclubin\nEurope.Meanwhile ,atth

In [ ]:
response = agent_executor.invoke("Mesélj nekem valamit a Bécsi állatvásárról")



> Entering new AgentExecutor chain...

Invoking: `ausztriai_kereso` with `{'query': 'Bécsi állatvásár'}`


www .khm.atMuseum of
Natur alHistory
1,Burgring 7
Tel.+431521770
Thu–Mon 09:00–18:00
Wed–Mon 09:00–20:00
www .nhm- wien.ac .at
Parliament
1,Dr.-Karl-Renner-Ring 3
Tel.+431401100
Mon, Tue,Wed,Fri
08:00–18:00
Thu08:00– 21:00
Sat09:00– 17:00
When plenar ysittings
takeplace until 21:00 (or
aslong assittings take)
www .parlament. gv.at
Volksgarten
1,Burgring
Mon– Sat07:00–17:30
(November –Mar ch)
www .bunde sgaer ten.at/
hofbur ggaer ten/
volksgarten
Burgtheater
1,Universitätsring 2
Tel.+431514444545
www .burgthe ater.at
City Hall
1,Friedrich- Schmidt-
Platz1
Tel.+43140004001
Mon–F ri07:30–17:00
(City ofVienna service)
www .wien.gv.at/english
University
ofVienna
1,Universitätsring 1
Tel.+43142770
Mon–F ri06:00– 22:00
Sat07:00–19:00
www .univie .ac.at

and vegetables,flowerstalls, long-e stablished mark etstallsand
afarmer s’marketonFridaysandSatur days,therearealso plenty
ofrestauran

In [ ]:
response

{'input': 'Mesélj nekem valamit a Bécsi állatvásárról',
 'chat_history': [HumanMessage(content='Mesélj nekem valamit a Bécsi állatvásárról'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Bécsi állatvásár"}', 'name': 'ausztriai_kereso'}}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 1925, 'total_tokens': 1952}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'function_call', 'logprobs': None}, id='run-41383b90-6940-4e77-be8a-ea3a79ca4bb6-0'),
  FunctionMessage(content='www .khm.atMuseum of\nNatur alHistory\n1,Burgring 7\nTel.+431521770\nThu–Mon 09:00–18:00\nWed–Mon 09:00–20:00\nwww .nhm- wien.ac .at\nParliament\n1,Dr.-Karl-Renner-Ring 3\nTel.+431401100\nMon, Tue,Wed,Fri\n08:00–18:00\nThu08:00– 21:00\nSat09:00– 17:00\nWhen plenar ysittings\ntakeplace until 21:00 (or\naslong assittings take)\nwww .parlament. gv.at\nVolksgarten\n1,Burgring\nMon– Sat07:00–17:30\n(November –Mar ch)\

In [ ]:
agent_executor({"input": "Milyen az indiai vendéglátás?"})



> Entering new AgentExecutor chain...
Az indiai vendéglátás híres a gazdag és ízletes ízekről, valamint a sokszínű konyháról. Az indiai ételek jellemzően fűszeresek és aromásak, sokféle fűszer és gyógynövény kerül felhasználásra az ételek elkészítése során. Az indiai konyha sokféle ételt kínál, mint például curryk, tandoori ételek, biryani, dál és számos más fogás. Az ételek gyakran vegetáriánusak, de megtalálhatók húsos ételek is, mint például a csirke vagy bárány. Az indiai vendéglátás fontos eleme a gasztronómiai élmény mellett a vendégszeretet és az otthonos hangulat teremtése is. A tradicionális indiai éttermek mellett ma már számos modern és stílusos indiai étterem is található világszerte, ahol az autentikus ízeket és kulináris élményeket kínálják a vendégeknek. Összességében az indiai vendéglátás gazdag és változatos kínálattal rendelkezik, amely sokak számára vonzó gasztronómiai élményt nyújt.

> Finished chain.


{'input': 'Milyen az indiai vendéglátás?',
 'chat_history': [AIMessage(content='A bécsi jelenkorban számos kulturális és zenei élmény várja az érdeklődőket. A Mozart-házban lehetőség van felfedezni Mozart eredeti, díszes lakását, ahol a nagy zeneszerző partikat rendezett 1784 és 1787 között. A Porgy&Bess jazzklubban élő zenét hallgathatunk, mely ma Európa legszebb jazzklubjának számít. A Ronacher színházban nagyszabású musical előadásokat tekinthetünk meg. A Hangok Háza zenei múzeum lehetőséget nyújt a hangok kísérletezésére és a karmesterpálca megforgatására. A városban található Johann Strauss emlékmű és Ludwig van Beethoven emlékmű is fontos látványosságok. A Wiener Konzerthausban számos esemény zajlik, ahol Bach-tól a Pop zenéig széles repertoárban található programok. Az Jugendstil stílusú épületek és az Otto Wagner által tervezett épületek is jellemzőek a városra, amelyek az osztrák art nouveau stílust képviselik. A városban sétálva felfedezhetjük a Jugendstil korszak művészeinek

#Kombináljunk össze több tool-t!

In [ ]:
from langchain import SerpAPIWrapper

search = SerpAPIWrapper(serpapi_api_key=userdata.get('SERPAPI_API_KEY'))
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="időjárási információk keresése"
    ),
    create_retriever_tool(
        db.as_retriever(),
        "ausztriai_kereso",
        "Ausztriára vonatkozó dokumentumok keresése és visszaküldése.",
    )
    ]
agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

In [ ]:
result = agent_executor.invoke("Foglald össze a bécsi jelenkort")



> Entering new AgentExecutor chain...

Invoking: `ausztriai_kereso` with `{'query': 'Bécs jelenkora'}`


WestartwiththeMozarthaus Vienna experience .
Itscente rpiec e,Mozart’sopulentoriginal apart-
ment onthegrandBeletageorpianonobile,
hostedmanyofthegreatcomp oser’sparties
betw een1784and1787.And nowtoamodern-
eravenue: anyoneinsearchofjazzinVienna
willstumble upon Porgy&Besssooner orlater,wher ethere’slive
music toenjoyvirtuallyeveryday.Oncealegenda ryeroticcinem a,
todayPorgyhasareputationasthemost beautifuljazzclubin
Europe.Meanwhile ,attheRonacher ,abeacon ofVienna ‘stheater
scene,it‘sshowtime forbigmusicalproduc tions.UpnextisVienna’s
sound museum, theHouse ofMusic .Dullmuseums areathing of
thepast –feelfreetojoininhere:wield theconducto r’sbaton,
experiment with sounds. Agreatplace forthewhole family!
Theshining high light andphot ohotspot intheStadtp arkisthe
gold enJohann Strauss memori al.Thewaltzking freque ntlyper-
formed intheKursalon right nexttoit.Just afewsteps farthe

In [ ]:
result['output']

'A fenti szöveg alapján a bécsi jelenkori helyzet összefoglalása a következőképpen történhet:\n\n- Bécs történelme során fontos szerepet játszott hídfőállásként, és a városnak jelentős szerepe volt a történelmi események során, például a török elleni védekezés és a második világháború idején.\n- Bécs több mint fél évszázada független a Habsburgoktól, és a városnak hasonlóságai vannak Budapesttel földrajzi elhelyezkedésében, történelmében és lakosságszámában.\n- A bécsi lakosság rosszkedvűnek és mogorvának tűnik, és haragot éreztek a magyarok iránt, mivel Budapest jobban élt a háború idején.\n- Bécsben különböző nemzetközi események és konferenciák zajlanak, amelyek vonzzák a turistákat és a világfigyelmet.\n- A városban kongresszusok és fesztiválok rendezése terén kiválóak, és rendelkeznek megfelelő helyszínekkel és infrastruktúrával.\n- A szövegben személyes emlékek is megjelennek a szerző Bécsben tett látogatásairól, amelyek a történelmi események és a város hangulatának bemutatására

In [ ]:
result = agent_executor.invoke("Milyen a jelenlegi időjárás Pécsen?")



> Entering new AgentExecutor chain...

Invoking: `Search` with `időjárás Pécs`


{'type': 'weather_result', 'temperature': '48', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '59%', 'wind': '13 mph', 'location': 'Pécs, Hungary', 'date': 'Sunday 9:00 AM', 'weather': 'Partly cloudy'}Jelenleg Pécsen 48 Fahrenheit fok van, részben felhős időjárás mellett. A csapadék valószínűsége 0%, a páratartalom pedig 59%. Az szélsebesség 13 mph. Ez az információ a jelenlegi időjárási viszonyokat mutatja.

> Finished chain.


In [ ]:
result['output']

'Jelenleg Pécsen 48 Fahrenheit fok van, részben felhős időjárás mellett. A csapadék valószínűsége 0%, a páratartalom pedig 59%. Az szélsebesség 13 mph. Ez az információ a jelenlegi időjárási viszonyokat mutatja.'

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", huggingfacehub_api_token=userdata.get('HF_TOKEN'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
agent_HF = initialize_agent(tools, llm = llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent_HF.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
 I don't have real-time data and I don't know the current population of Canada. I need to find a reliable source with this information.
Action: ausztriai_kereso
Action Input: Canada population 2023
Observation: #ViennaNo w
vienna.inf o
route
Ways
toexplore
thecity.

Belle vuewiese
19,Himmelstrasse 115
iU3VolkstheaterU3StubentorU1,U3
StephansplatzU3HerrengasseU1,U4
SchwedenplatzU2SchottentorU2,U4
SchottenringWA
SA
-
GA
SS
E
BÄCKERSTRASSEFREYUNG
DOROTHEERG.
SINGERSTR.
WI
CK
ENB
UR
GGA
SS
E
EBENDORFERSTRASSE
S.-F.-G.
SC
HL
ÖS
SEL
GA
SS
E
TUCHLAUBENRATHAUSSTRASSE
HABSBURGERG.
WEIHBURGGA
SS
EROTENTURMSTR.GA
RN
ISONGA
SS
E
FLEISCHMARKTRENNGASSE
SEILERG.AM
HOF
STRAUCHG.
SPIEGELG.GR
AB
ENN.-G.
KÄRNTNERSTRASSEDOMG
.
BLUTG.KOHLMARKTALSERSTRASSE
UNIVERSITÄTSRINGSCHOTTENRINGLIEC
HT
ENS
TE
IN
-
STR
AS
SEWÄ
HRING
ER
STR
AS
SE
LÖ
W
EN
-STR.TIEFERGRABEN
HE
RRE
NG
.R.-G
.TEINFALTSTR.SC
HOT
TE
NG
AS
SE
SALZG.UNIVERSITÄTSSTRASSEFRANKHPL.O.-W.-PL.SCHWARZSPANIERSTRAS

"I'm sorry, I couldn't find the information about Canada's population in 2023 in English."

In [ ]:
agent_HF.run("Milyen a jelenlegi időjárás Pécsen?")



> Entering new AgentExecutor chain...
 I don't know the current weather in Pécs. I need to use the search tool to find out.
Action: Search
Action Input: Current weather in Pécs, Hungary
Observation: {'type': 'weather_result', 'temperature': '47', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '56%', 'wind': '11 mph', 'location': 'Pécs, Hungary', 'date': 'Sunday 10:00 AM', 'weather': 'Cloudy'}
Thought: I now know the final answer
Final Answer: The current weather in Pécs is cloudy with a temperature of 47 Fahrenheit, 0% precipitation, 56% humidity, 11 mph wind speed, and the date is Sunday at 10:00 AM.

> Finished chain.


'The current weather in Pécs is cloudy with a temperature of 47 Fahrenheit, 0% precipitation, 56% humidity, 11 mph wind speed, and the date is Sunday at 10:00 AM.'

In [ ]:
agent_HF.run("How many people lived in China in 2023?")



> Entering new AgentExecutor chain...
 I don't have this information, I need to search for it.
Action: Search
Action Input: Number of people living in China in 2023
Observation: around 1.41 billion people
Thought: I now know the final answer
Final Answer: Around 1.41 billion people lived in China in 2023.

> Finished chain.


'Around 1.41 billion people lived in China in 2023.'

In [ ]:
agent_HF.run("Milyen látványosságok vannak Bécsben?")



> Entering new AgentExecutor chain...
 I need to translate this question to English and then search for the answer. The question is asking for tourist attractions in Vienna.
Action: Translate
Action Input: Milyen látványosságok vannak Bécsben?
Observation: Translate is not a valid tool, try one of [Search].
Thought: I made a mistake, I should use the Search tool to find tourist attractions in Vienna.
Action: Search
Action Input: tourist attractions in Vienna
Observation: [{'position': 1, 'block_position': 'top', 'title': 'Vienna Old Town Walking Tour, Hofburg, Spanish Riding School', 'price': '$25', 'extracted_price': 25, 'link': 'https://www.getyourguide.com/', 'source': 'GetYourGuide', 'rating': 4.8, 'reviews': 10, 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRx7yRKuqNePKK0z2gnVEECbmmEh0Mw6glerf06xi_w9Duv&s=4', 'extensions': ['Free cancellation']}, {'position': 2, 'block_position': 'top', 'title': 'The best of Vienna on foot', 'price': '$31', 'extracted_price

'There are many tourist attractions in Vienna, including the Vienna Old Town Walking Tour, Hofburg, Spanish Riding School, the best of Vienna on foot tour, and a private walking tour with a local guide. These tours offer a range of prices and experiences, with options for free cancellation.'

#Általános célú tool egy LLM

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=OpenAI(api_key = userdata.get('openai_api_key')), prompt=prompt)

# initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='use this tool for general purpose queries and logic'
)

In [ ]:
tools.append(llm_tool)

# reinitialize the agent
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=OpenAI(api_key = userdata.get('openai_api_key')),
    verbose=True,
    max_iterations=3
)

In [ ]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
 This is a general knowledge question.
Action: Language Model
Action Input: "what is the capital of Norway?"
Observation: 

The capital of Norway is Oslo.
Thought: I now know the final answer.
Final Answer: Oslo

> Finished chain.


{'input': 'what is the capital of Norway?', 'output': 'Oslo'}